In [2]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from utils import forward_tracer, backward_tracer, Char2Vec, num_flat_features

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np

from tqdm import tqdm

from IPython.display import clear_output

In [3]:
torch.nn

<module 'torch.nn' from '/home/ge/anaconda3/envs/deep-learning/lib/python3.6/site-packages/torch/nn/__init__.py'>

In [33]:
?nn.GRU

In [25]:
rnn = nn.GRU(10, 20, 2)
input = Variable(torch.randn(5, 3, 10))
h0 = Variable(torch.randn(2, 3, 20))
output, hn = rnn(input, h0)
print(output.size())

torch.Size([5, 3, 20])


In [32]:
class GruRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GruRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.encoder = nn.Linear(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, 1)
        self.decoder = nn.Linear(hidden_size, output_size)
        self.softmax = F.softmax
        
    def forward(self, x, hidden):
        embeded = self.encoder(x)
        gru_output, hidden = self.gru(embeded.view(-1, 1, self.input_size), hidden.view(1, 1, -1))
        output = self.decoder(gru_output.view(-1, self.hidden_size))
        return output, hidden

input_size = 101
hidden_size = 101
output_size = 101

gRNN = GruRNN(input_size, hidden_size, output_size)

gRNN(Variable(torch.FloatTensor(10000, 101)),
     Variable(torch.FloatTensor(101)))

(Variable containing:
 -0.0958 -0.0411 -0.0050  ...   0.0697 -0.0073 -0.0698
 -0.0875 -0.0418 -0.0105  ...   0.0676 -0.0271 -0.0742
 -0.0798 -0.0431 -0.0150  ...   0.0665 -0.0374 -0.0760
           ...             ⋱             ...          
 -0.0676 -0.0447 -0.0241  ...   0.0642 -0.0454 -0.0773
 -0.0676 -0.0447 -0.0241  ...   0.0642 -0.0454 -0.0773
 -0.0676 -0.0447 -0.0241  ...   0.0642 -0.0454 -0.0773
 [torch.FloatTensor of size 10000x101], Variable containing:
 ( 0 ,.,.) = 
 
 Columns 0 to 8 
    0.1587 -0.0741  0.0693 -0.1123  0.1256  0.0301  0.0243  0.0200  0.0807
 
 Columns 9 to 17 
    0.0037 -0.0817  0.0118 -0.0359  0.0019  0.0554 -0.0608 -0.0108 -0.0539
 
 Columns 18 to 26 
   -0.0195  0.0433 -0.0090  0.1201 -0.1222  0.0183 -0.0192 -0.0266 -0.0782
 
 Columns 27 to 35 
    0.0480  0.0021 -0.0520 -0.0773 -0.1464 -0.0051  0.0316 -0.0374  0.0037
 
 Columns 36 to 44 
    0.0061  0.1132  0.0179 -0.0742 -0.0382 -0.0021 -0.0242 -0.0183 -0.0139
 
 Columns 45 to 53 
    0.1005 -0.0213 -

In [18]:
?nn.Linear

In [15]:
gRNN(Variable(torch.FloatTensor(10000, 101)),
     Variable(torch.FloatTensor(1, 10, 101)))

RuntimeError: inconsistent tensor size at /data/users/soumith/miniconda2/conda-bld/pytorch-cuda80-0.1.10_1488758793045/work/torch/lib/TH/generic/THTensorMath.c:827